In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1588472063188_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl (10.5 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [3]:
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Github Commands 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

invalid syntax (<stdin>, line 1)
  File "<stdin>", line 1
    !git pull origin master
    ^
SyntaxError: invalid syntax



# Sample Data

In [ ]:
input_bucket = 's3://amazon-reviews-pds'
input_path = '/parquet/product_category=Books/*.parquet'
df = spark.read.parquet(input_bucket + input_path)

In [9]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   15444933|R1WWG70WK9VUCH|1848192576|     835940987|Standing Qigong f...|          5|            9|         10|   N|                Y|Informative AND i...|After attending a...| 2015-05-02|2015|
|         US|   20595117|R1EQ3POS0RIOD5|145162445X|     574044348|A Universe from N...|          4|            4|          7|   N|                N|Between 'Nothing'...

In [3]:
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(marketplace='US', customer_id='15444933', review_id='R1WWG70WK9VUCH', product_id='1848192576', product_parent='835940987', product_title='Standing Qigong for Health and Martial Arts - Zhan Zhuang', star_rating=5, helpful_votes=9, total_votes=10, vine='N', verified_purchase='Y', review_headline='Informative AND interesting!', review_body="After attending a few Qigong classes, I wanted to have a book to read and re-read the instructions so I could practice at home.  I also wanted to gain more of an understanding of the purpose and benefit of the movements in order to practice them with a more focused purpose.<br /><br />The book exceeded my expectations.  The explanations are very clear and are paired with photos showing the correct form.  The book itself is more than just the Qigong, it's a very interesting read.  I read the whole book in two days and will read it again. I rarely read books twice!  The book has provided the information and additional instruction that I was looking f

In [4]:
df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, year: string]

In [6]:
df.count(
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20726160

# Import Real Data from S3  

In [5]:
path =  's3://bosch-data-spark/data/train_numeric.csv'
df_train_numeric = spark.read.load(path,format="csv",inferSchema = True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
len(df_train_numeric.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

970

In [11]:
df_train_numeric.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183747

In [12]:
df_train_numeric.select(df_train_numeric[-1]).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [13]:
path =  's3://bosch-data-spark/data/train_categorical.csv'
df_train_categorical = spark.read.load(path,format="csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_train_categorical.limit(100).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Id L0_S1_F25 L0_S1_F27  ... L3_S49_F4237 L3_S49_F4239 L3_S49_F4240
0     4      None      None  ...         None         None         None
1     6      None      None  ...         None         None         None
2     7      None      None  ...         None         None         None
3     9      None      None  ...         None         None         None
4    11      None      None  ...         None         None         None
..  ...       ...       ...  ...          ...          ...          ...
95  192      None      None  ...         None         None         None
96  197      None      None  ...         None         None         None
97  200      None      None  ...         None         None         None
98  201      None      None  ...         None         None         None
99  205      None      None  ...         None         None         None

[100 rows x 2141 columns]

In [17]:
df_train_numeric.select(df_train_numeric[0]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
| Id|
+---+
|  4|
|  6|
|  7|
|  9|
| 11|
+---+
only showing top 5 rows

In [50]:
df_train_categorical.select(df_train_categorical[0]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
| Id|
+---+
|  4|
|  6|
|  7|
|  9|
| 11|
+---+
only showing top 5 rows

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# NOTE: On Join for PySpark, indicate the column match as a string rather than the commented out statement 

In [39]:
# inner_join = df_train_numeric.join(df_train_categorical, df_train_numeric.Id == df_train_categorical.Id )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# On Exploring the Categorical Dataset, it appears there are so many NaN's that it simply doesn't make sense to analyze it

In [37]:
# inner_join = df_train_numeric.join(df_train_categorical, "Id" )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
# len(inner_join.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# count all nan values for all columns
# inner_join.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in inner_join.columns]).show()

In [ ]:
df_train_numeric.head(1)[:10]

In [100]:
def df_wrangle(df):
    df =  df.drop("Id","Response")
    df = df.select([col(c).cast("double") for c in df.columns])
    imputer = Imputer(
    inputCols=df.columns, 
    outputCols=["{}_imputed".format(c) for c in df.columns]
    )
    df = imputer.fit(df).transform(df)
    df = df[imputer.getOutputCols()]
    return df 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def df_preprocess(df):
    assembler = VectorAssembler(
    inputCols=[df.columns],
    outputCol='features'
    )
    

In [101]:
df_train_numeric = df_wrangle(df_train_numeric)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [102]:
len(df_train_numeric.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

968

In [112]:
df_train_numeric.select(df_train_numeric.columns[:10]).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   L0_S0_F0_imputed  L0_S0_F2_imputed  ...  L0_S0_F16_imputed  L0_S0_F18_imputed
0          0.030000         -0.034000  ...       2.000000e-02           0.083000
1          0.000055          0.000091  ...       6.722445e-07           0.000005
2          0.088000          0.086000  ...      -2.250000e-01          -0.147000
3         -0.036000         -0.064000  ...      -2.600000e-02          -0.046000
4         -0.055000         -0.086000  ...      -1.690000e-01          -0.099000

[5 rows x 10 columns]

In [ ]:
spark.sql("SELECT C* FROM table1 where ")